# Compiling different sources of sports data

In [52]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import datetime
from fp.fp import FreeProxy

## Football: Chicago Bears

In [25]:
# URL of the webpage containing the NFL 2018 season data
response_football = requests.get("https://www.pro-football-reference.com/teams/chi/2018.htm")

soup = BeautifulSoup(response_football.content, "html.parser")
events = soup.find("table",id="games").tbody.findAll("tr")

row_constructor = {"Date": [], "Time": [], "Duration": [], "Stadium": []}

for event in events:
    try:
        boxscorelink = "https://www.pro-football-reference.com/" + event.find("td",{"data-stat":"boxscore_word"}).a["href"]
        print(boxscorelink)
        moreInfo = BeautifulSoup(requests.get(boxscorelink).content,"html.parser")
        stadium = moreInfo.find("div",class_="scorebox_meta").findAll("div")[2].text
        print(stadium,stadium == "Soldier Field")
        if(stadium == "Soldier Field"):
            row_constructor["Date"].append(event.find("td",{"data-stat":"game_date"}).text)
            print(event.find("td",{"data-stat":"game_date"}).text)
            row_constructor["Time"].append(event.find("td",{"data-stat":"gametime"}).text)
            row_constructor["Duration"].append(moreInfo.find("div",class_="scorebox_meta").findAll("div")[4].text)
            row_constructor["Stadium"].append(stadium)
    except TypeError:
        print("Error parsing. Row has no boxscore.")
row_constructor

AttributeError: 'NoneType' object has no attribute 'tbody'

In [ ]:
nfldf = pd.DataFrame.from_dict(row_constructor)
nfldf

## Baseball: Chicago Cubs & Chicago White Sox

In [63]:
#This takes a minute unfortunately. We have to send a seperate request each time we want to know the game duration. 
#I dont know why but there are no games found prior to April 5th.

data = requests.get("http://statsapi.mlb.com/api/v1/schedule/games/?sportId=1&startDate=2018-03-29&endDate=2018-09-30").json()

list_cubs = []
list_whitesox = []
counter = 0

for datum in data["dates"]:
    for game in datum["games"]:
        name = game["teams"]["home"]["team"]["name"]
        if(name == "Chicago Cubs"):
            print("Found Cubs home game. Opponent: ", game["teams"]["away"]["team"]["name"])
            counter += 1
            gamedata_cubs = requests.get("http://statsapi.mlb.com" + game["link"]).json()    
            temp1 = [game["gameDate"],gamedata_cubs["gameData"]["gameInfo"]["gameDurationMinutes"]]
            list_cubs.append(temp1)
        elif(name == "Chicago White Sox"):
            print("Found White Sox home game. Opponent: ", game["teams"]["away"]["team"]["name"])
            counter += 1
            gamedata_whitesox = requests.get("http://statsapi.mlb.com" + game["link"]).json()
            temp2 = [game["gameDate"],gamedata_whitesox["gameData"]["gameInfo"]["gameDurationMinutes"]]
            list_whitesox.append(temp2)
        if(counter > 10):
            break;
    if(counter > 10):
            break;
            
Chicago_Cubs_temp = pd.DataFrame(list_cubs)
Chicago_White_Sox_temp = pd.DataFrame(list_whitesox) 

Found White Sox home game. Opponent:  Detroit Tigers
Found White Sox home game. Opponent:  Detroit Tigers
Found White Sox home game. Opponent:  Detroit Tigers
Found White Sox home game. Opponent:  Tampa Bay Rays
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found White Sox home game. Opponent:  Tampa Bay Rays
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found White Sox home game. Opponent:  Tampa Bay Rays
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found Cubs home game. Opponent:  Pittsburgh Pirates
Found Cubs home game. Opponent:  Atlanta Braves


In [64]:
#Chicago has an offset of GMT-5. We don't have to mess around with daylight saving time, because the switch does not occur during the season.
Chicago_Cubs_df = pd.DataFrame()
Chicago_Cubs_df["Game_Start"] = pd.to_datetime(Chicago_Cubs_temp[0]) - pd.to_timedelta(5, unit="h")
Chicago_Cubs_df["Game_End"] = Chicago_Cubs_df["Game_Start"] + pd.to_timedelta(Chicago_Cubs_temp[1], unit="m")

Chicago_White_Sox_df = pd.DataFrame()
Chicago_White_Sox_df["Game_Start"] = pd.to_datetime(Chicago_White_Sox_temp[0]) - pd.to_timedelta(5, unit="h")
Chicago_White_Sox_df["Game_End"] = Chicago_White_Sox_df["Game_Start"] + pd.to_timedelta(Chicago_White_Sox_temp[1], unit="m")

In [65]:
Chicago_White_Sox_df

,Game_Start,Game_End
0,2018-04-05 15:10:00+00:00,2018-04-05 19:13:00+00:00
1,2018-04-07 13:10:00+00:00,2018-04-07 16:49:00+00:00
2,2018-04-08 13:10:00+00:00,2018-04-08 15:48:00+00:00
3,2018-04-09 13:10:00+00:00,2018-04-09 16:57:00+00:00
4,2018-04-10 13:10:00+00:00,2018-04-10 16:54:00+00:00
5,2018-04-11 13:10:00+00:00,2018-04-11 16:12:00+00:00


## Chicago Blackhawks

In [54]:
# URL of the API containing the NHL 2017/2018 season data for the Chicago Blackhawks

response_hockey = requests.get("https://api-web.nhle.com/v1/club-schedule-season/CHI/20172018")
data = response_hockey.json()

nhldf_temp = pd.DataFrame(data["games"])
nhldf_temp

,id,season,gameType,gameDate,venue,neutralSite,startTimeUTC,easternUTCOffset,venueUTCOffset,venueTimezone,gameState,gameScheduleState,tvBroadcasts,awayTeam,homeTeam,periodDescriptor,gameOutcome,gameCenterLink
0,2017010015,20172018,1,2017-09-19,{'default': 'Nationwide Arena'},False,2017-09-19T23:00:00Z,-04:00,-04:00,US/Eastern,FINAL,OK,"[{'id': 107, 'market': 'N', 'countryCode': 'US...","{'id': 16, 'placeName': {'default': 'Chicago'}...","{'id': 29, 'placeName': {'default': 'Columbus'...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},/gamecenter/chi-vs-cbj/2017/09/19/2017010015
1,2017010034,20172018,1,2017-09-21,{'default': 'United Center'},False,2017-09-22T00:30:00Z,-04:00,-05:00,America/Chicago,FINAL,OK,"[{'id': 312, 'market': 'N', 'countryCode': 'US...","{'id': 17, 'placeName': {'default': 'Detroit'}...","{'id': 16, 'placeName': {'default': 'Chicago'}...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},/gamecenter/det-vs-chi/2017/09/21/2017010034
2,2017010051,20172018,1,2017-09-23,{'default': 'United Center'},False,2017-09-24T00:30:00Z,-04:00,-05:00,America/Chicago,FINAL,OK,"[{'id': 312, 'market': 'N', 'countryCode': 'US...","{'id': 29, 'placeName': {'default': 'Columbus'...","{'id': 16, 'placeName': {'default': 'Chicago'}...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},/gamecenter/cbj-vs-chi/2017/09/23/2017010051
3,2017010067,20172018,1,2017-09-25,{'default': 'TD Garden'},False,2017-09-25T23:00:00Z,-04:00,-04:00,US/Eastern,FINAL,OK,"[{'id': 128, 'market': 'H', 'countryCode': 'US...","{'id': 16, 'placeName': {'default': 'Chicago'}...","{'id': 6, 'placeName': {'default': 'Boston'}, ...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},/gamecenter/chi-vs-bos/2017/09/25/2017010067
4,2017010087,20172018,1,2017-09-28,{'default': 'Little Caesars Arena'},False,2017-09-28T23:30:00Z,-04:00,-04:00,America/Detroit,FINAL,OK,"[{'id': 107, 'market': 'N', 'countryCode': 'US...","{'id': 16, 'placeName': {'default': 'Chicago'}...","{'id': 17, 'placeName': {'default': 'Detroit'}...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},/gamecenter/chi-vs-det/2017/09/28/2017010087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2017021196,20172018,2,2018-03-29,{'default': 'United Center'},False,2018-03-30T00:30:00Z,-04:00,-05:00,America/Chicago,OFF,OK,"[{'id': 292, 'market': 'A', 'countryCode': 'CA...","{'id': 52, 'placeName': {'default': 'Winnipeg'...","{'id': 16, 'placeName': {'default': 'Chicago'}...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},/gamecenter/wpg-vs-chi/2018/03/29/2017021196
84,2017021203,20172018,2,2018-03-30,{'default': 'Pepsi Center'},False,2018-03-31T01:00:00Z,-04:00,-06:00,America/Denver,OFF,OK,"[{'id': 47, 'market': 'H', 'countryCode': 'US'...","{'id': 16, 'placeName': {'default': 'Chicago'}...","{'id': 21, 'placeName': {'default': 'Colorado'...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},/gamecenter/chi-vs-col/2018/03/30/2017021203
85,2017021239,20172018,2,2018-04-04,{'default': 'Scottrade Center'},False,2018-04-05T00:00:00Z,-04:00,-05:00,US/Central,OFF,OK,"[{'id': 241, 'market': 'N', 'countryCode': 'US...","{'id': 16, 'placeName': {'default': 'Chicago'}...","{'id': 19, 'placeName': {'default': 'St. Louis...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},/gamecenter/chi-vs-stl/2018/04/04/2017021239
86,2017021255,20172018,2,2018-04-06,{'default': 'United Center'},False,2018-04-07T00:30:00Z,-04:00,-05:00,America/Chicago,OFF,OK,"[{'id': 13, 'market': 'A', 'countryCode': 'US'...","{'id': 19, 'placeName': {'default': 'St. Louis...","{'id': 16, 'placeName': {'default': 'Chicago'}...","{'periodType': 'REG', 'maxRegulationPeriods': 3}",{'lastPeriodType': 'REG'},/gamecenter/stl-vs-chi/2018/04/06/2017021255


In [116]:
# Create mask for homeTeam ID
mask_home_team = nhldf_temp["homeTeam"].apply(lambda x: x['id']) == 16

# Create mask for gameDate
mask_game_date = pd.to_datetime(nhldf_temp["gameDate"]) >= pd.to_datetime("2018-01-01")

mask_combined = mask_home_team & mask_game_date

# Filter dataframe using mask
nhldf = nhldf_temp[mask_combined].copy()

#Unfortunately the API does not offer information about the duration of the game. We will use 2 hours and 19 minutes as average value.
nhldf.loc[:, 'venueUTCOffset'] = nhldf['venueUTCOffset'].apply(lambda x: int(x[:3]) + int(x[4:6])/60)
nhldf.loc[:, 'Game_Start'] = pd.to_datetime(nhldf["startTimeUTC"]) + pd.to_timedelta(nhldf["venueUTCOffset"], unit="h")
nhldf["Game_End"] = nhldf["Game_Start"] + pd.Timedelta(hours=2, minutes=19) 
nhldf[["Game_Start","Game_End"]] 

,Game_Start,Game_End
45,2018-01-05 19:30:00+00:00,2018-01-05 21:49:00+00:00
46,2018-01-07 14:00:00+00:00,2018-01-07 16:19:00+00:00
48,2018-01-10 19:00:00+00:00,2018-01-10 21:19:00+00:00
49,2018-01-12 19:30:00+00:00,2018-01-12 21:49:00+00:00
50,2018-01-14 11:30:00+00:00,2018-01-14 13:49:00+00:00
51,2018-01-20 19:30:00+00:00,2018-01-20 21:49:00+00:00
52,2018-01-22 19:30:00+00:00,2018-01-22 21:49:00+00:00
53,2018-01-24 19:00:00+00:00,2018-01-24 21:19:00+00:00
58,2018-02-06 19:30:00+00:00,2018-02-06 21:49:00+00:00
59,2018-02-08 19:30:00+00:00,2018-02-08 21:49:00+00:00


## Chicago Fire FC

In [143]:
# URL of the Wikipedia page

response = requests.get("https://en.wikipedia.org/wiki/2018_Chicago_Fire_season")

soup = BeautifulSoup(response.content, "html.parser")
events = soup.find_all("table", class_="tmpl-football-box-collapsible")

row_constructor = {"Date": [], "Stadium": []}

for event in events:
    
    tr = event.tbody.find_all("tr")
    upper = tr[0].find_all("td")
    lower = tr[1].find_all("td")
    
    date = upper[0].find("span").text 
    time = lower[0].text
    try:
        datetime_obj = datetime.datetime.strptime(f"{date} {time}", "%B %d, %Y %H:%M")
        row_constructor["Date"].append(datetime_obj)
        row_constructor["Stadium"].append(lower[4].find("span").text)
    except ValueError:
        print(f"Error parsing time for event: {date}")

msldf = pd.DataFrame.from_dict(row_constructor)
msldf = msldf[msldf["Stadium"] == "Toyota Park"]
msldf

Error parsing time for event: February 17, 2018
Error parsing time for event: October 21, 2018
Error parsing time for event: October 28, 2018
Error parsing time for event: August 8, 2018


,Date,Stadium
4,2018-03-03 12:00:00,Toyota Park
5,2018-03-10 17:00:00,Toyota Park
7,2018-03-31 17:00:00,Toyota Park
8,2018-04-07 19:30:00,Toyota Park
9,2018-04-14 14:30:00,Toyota Park
12,2018-05-05 17:00:00,Toyota Park
13,2018-05-09 19:30:00,Toyota Park
15,2018-05-20 15:00:00,Toyota Park
18,2018-06-02 19:30:00,Toyota Park
19,2018-06-09 19:30:00,Toyota Park


## Chicago Bulls

In [67]:
#https://www.basketball-reference.com/teams/CHI/2018_games.html
#https://www.basketball-reference.com/teams/CHI/2019_games.html

response_nba2018 = requests.get("https://www.basketball-reference.com/teams/CHI/2018_games.html",proxies= {"https":FreeProxy(https=True).get()},verify=False)
response_nba2019 = requests.get("https://www.basketball-reference.com/teams/CHI/2019_games.html",proxies= {"https":FreeProxy(https=True).get()},verify=False)
soup2018 = BeautifulSoup(response_nba2018.content, "html.parser")
soup2019 = BeautifulSoup(response_nba2019.content, "html.parser")
print(soup2018)
events2018 = soup2018.find("table",id="games").tbody.findAll("tr")
events2019 = soup2019.find("table",id="games").tbody.findAll("tr")
events = events2018[37:] + events2019[:37] #Only consider 2018 games
row_constructor = {"Date": [], "Time": [], "Stadium": []}

for event in events:
    if(str(event.get("class")) == "['thead']"):
        continue
    boxscorelink = "https://www.basketball-reference.com" + event.find("td",{"data-stat":"box_score_text"}).a["href"]
    boxscoreRequest = requests.get(boxscorelink,proxies= {"https":FreeProxy(https=True).get()},verify=False)
    moreInfo = BeautifulSoup(boxscoreRequest.content,"html.parser")
    
    stadium = moreInfo.find("div",class_="scorebox_meta").findAll("div")[1].text
    if(stadium == "United Center, Chicago, Illinois"):
        row_constructor["Date"].append(event.find("td",{"data-stat":"date_game"}).text)
        row_constructor["Time"].append(event.find("td",{"data-stat":"game_start_time"}).text)
        row_constructor["Stadium"].append(stadium)

row_constructor

C:\Users\marlo\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '185.217.136.67'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


FreeProxyException: There are no working proxies at this time.